## Romi examples - 2022 0131 CJH
---
* https://docs.wpilib.org/en/stable/docs/romi-robot/getting-to-know-romi.html
* The romi is different in the way it runs
  * Unlike the robot, it does not run on the roborio (usually 10.0.0.2)
  * The devs trick it to run on the driver station with the --ws-client flag makes the sim look for the romi 
    * your laptop has one address, typically 10.0.0.100 - 10.0.0.200)
    * the pi is 10.0.0.2, which is backwards from what is on the robot
* coprocessing on the romi is a bit different
  * you still run vision, etc on the pi but now network tables is served by the driver station, **not** the robot
* because everything is happening in the sim, it is not a good idea to draw the romi in the sim window

#### LEDs and Buttons on the romi
The romi webpage (10.0.0.2) lets you configure these, but by default
* DIO 0 is set to input, and it is the Button A on the romi
* DIO 3 is set to output, and it is the yellow LED on the romi
* DIO 1 can be configured to input (Button B) or output (green LED) - it's the first parameter of OnBoardIO's constructor
* DIO 2 can be configured to input (Button C) or output (red LED) - it's the second parameter of OnBoardIO's constructor

In [12]:
# excerpt from onboard io
import enum, typing
import commands2
import wpilib


class ChannelMode(enum.Enum):
    INPUT = 1
    OUTPUT = 2


class OnBoardIO(commands2.SubsystemBase):
    """This class represents the onboard IO of the Romi reference robot. This includes the pushbuttons
    and LEDs.
    DIO 0 - Button A (input only) DIO 1 - Button B (input) or Green LED (output) DIO 2 - Button C
    (input) or Red LED (output) DIO 3 - Yellow LED (output only)
    """

    MESSAGE_INTERVAL = 1.0

    def __init__(self, dio1: ChannelMode, dio2: ChannelMode) -> None:
        """Constructor.
        :param dio1: Mode for DIO 1 (input = Button B, output = green LED)
        :param dio2: Mode for DIO 2 (input = Button C, output = red LED)
        """
        super().__init__()

        self.nextMessageTime = 0

        self.buttonA = wpilib.DigitalInput(0)
        self.buttonB: typing.Optional[wpilib.DigitalInput] = None
        self.buttonC: typing.Optional[wpilib.DigitalInput] = None

        self.yellowLed = wpilib.DigitalOutput(3)
        self.greenLed: typing.Optional[wpilib.DigitalOutput] = None
        self.redLed: typing.Optional[wpilib.DigitalOutput] = None
        
    def getButtonAPressed(self) -> bool:
        """Gets if the A button is pressed."""
        return self.buttonA.get()
    
    def setYellowLed(self, value: bool) -> None:
        """Sets the yellow LED."""
        self.yellowLed.set(value)

In [11]:
# Example of how to use the onboard IO to sense a button press - somthing similar goes in robotcontainer
# I took out the selfs
import commands2.button
onboardIO = OnBoardIO(ChannelMode.INPUT, ChannelMode.OUTPUT)

onboardButtonA = commands2.button.Button(onboardIO.getButtonAPressed)
onboardButtonA.whenActive(
    commands2.PrintCommand("Button A Pressed")
).whenInactive(commands2.PrintCommand("Button A Released"))